In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
#working actual directory
df = pd.read_csv(r'A:\GIT\MUSER\muser-firebase-export\target\music-analysis.csv',encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 56 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   user_id                                     29 non-null     object 
 1   record_id                                   29 non-null     object 
 2   player_event_type                           16 non-null     object 
 3   ui_event_type                               12 non-null     object 
 4   event_current_time_in_milliseconds          28 non-null     float64
 5   event_nano_time                             28 non-null     float64
 6   event_seek_position_in_milliseconds         28 non-null     float64
 7   event_start_time                            0 non-null      float64
 8   event_elapsed_time                          0 non-null      float64
 9   song_album_id                               27 non-null     float64
 10  song_album_name 

In [3]:
#sorting the current timestamp in ascending order
df.sort_values('event_current_time_in_milliseconds', ascending=True, inplace=True)

In [4]:
#adding a new column event_current_time_ms_to_date to the csv 
df['event_current_time_ms_to_date'] = pd.to_datetime(df['event_current_time_in_milliseconds'], unit='ms')

In [5]:
#remove the empty valued audio_volume_data column
df = df.dropna(axis=0, subset=['audio_volume_data'])
df.shape

(28, 57)

In [6]:
#function to extract minimum volume from an array of audio_volume_data
def pull_vol_min(i):
    a,b,c,d= i.split(',')
    return c

In [7]:
df['min_volume'] =df['audio_volume_data'].apply(pull_vol_min)
df['min_volume'] = df['min_volume'].str.split('=').str[1]

In [8]:
#function to extract current volume from an array of audio_volume_data
def pull_vol_curr(i):
    a,b,c,d,e= i.split()
    start = b.find('[')
    end = b.find(',')
    substring = b[start+1:end]
    return substring

In [9]:
df['curent_volume'] = df.audio_volume_data.apply(pull_vol_curr)
df['curent_volume'] = df['curent_volume'].str.split('=').str[1]

In [10]:
df.head()

,user_id,record_id,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_nano_time,event_seek_position_in_milliseconds,event_start_time,event_elapsed_time,song_album_id,...,device_info_google_play_services_lib,device_info_ignoring_battery_optimizations,device_info_power_save_mode_enabled,device_info_sdk_version,device_info_sdk_version_int,device_info_talk_back_enabled,device_info_timestamp,event_current_time_ms_to_date,min_volume,curent_volume
16,Q5D01u10sfOs4gOLpgdsbmlrk8q2,0-a335579d-23a8-43fc-aa59-2fef8c89feb3,NaN,VOLUME_DOWN,1.630264e+12,2.879892e+13,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-29 19:06:12.167,0,8
17,Q5D01u10sfOs4gOLpgdsbmlrk8q2,1-da4b4f93-1619-4acb-b528-639c68df1d42,NaN,PLAY,1.630264e+12,2.882307e+13,0.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-29 19:06:36.321,0,8
5,Q5D01u10sfOs4gOLpgdsbmlrk8q2,2-575fd343-a771-448e-9fcd-ad5c686ef5e6,PLAY,NaN,1.630264e+12,2.882324e+13,0.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-29 19:06:36.486,0,8
12,Q5D01u10sfOs4gOLpgdsbmlrk8q2,3-f29bdaba-924f-44fb-ae29-478c4c072135,PLAY,NaN,1.630264e+12,2.904491e+13,0.0,NaN,NaN,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-29 19:10:18.159,0,8
25,Q5D01u10sfOs4gOLpgdsbmlrk8q2,4-12388d9e-dd3b-4b86-964b-3258f79a754f,NaN,PLAY,1.630264e+12,2.904522e+13,433.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-08-29 19:10:18.472,0,8


In [11]:
#filtering out the records that are greater than the minimum volume value
df= df[ df['curent_volume'] > df['min_volume'] ]

In [12]:
# shorten the final file by only keeping required columns in the final csv file
user_behavior_data = df[["user_id", "record_id", "song_id", "song_name", "player_event_type", "ui_event_type",  "event_current_time_in_milliseconds", "event_seek_position_in_milliseconds", "song_album_name", "song_artist_name", "song_duration", "song_name", "audio_volume_data","event_current_time_ms_to_date", "min_volume", "curent_volume"]]

In [13]:
user_behavior_data

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,min_volume,curent_volume
16,Q5D01u10sfOs4gOLpgdsbmlrk8q2,0-a335579d-23a8-43fc-aa59-2fef8c89feb3,NaN,NaN,NaN,VOLUME_DOWN,1.630264e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:06:12.167,0,8
17,Q5D01u10sfOs4gOLpgdsbmlrk8q2,1-da4b4f93-1619-4acb-b528-639c68df1d42,142.0,When You Wish Upon A Star,NaN,PLAY,1.630264e+12,0.0,Dream With Me,Jackie Evancho,221440.0,When You Wish Upon A Star,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:06:36.321,0,8
5,Q5D01u10sfOs4gOLpgdsbmlrk8q2,2-575fd343-a771-448e-9fcd-ad5c686ef5e6,142.0,When You Wish Upon A Star,PLAY,NaN,1.630264e+12,0.0,Dream With Me,Jackie Evancho,221440.0,When You Wish Upon A Star,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:06:36.486,0,8
12,Q5D01u10sfOs4gOLpgdsbmlrk8q2,3-f29bdaba-924f-44fb-ae29-478c4c072135,143.0,Nella Fantasia,PLAY,NaN,1.630264e+12,0.0,Dream With Me,Jackie Evancho,258534.0,Nella Fantasia,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:10:18.159,0,8
25,Q5D01u10sfOs4gOLpgdsbmlrk8q2,4-12388d9e-dd3b-4b86-964b-3258f79a754f,109.0,Remedy,NaN,PLAY,1.630264e+12,433.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:10:18.472,0,8
13,Q5D01u10sfOs4gOLpgdsbmlrk8q2,5-c193f0ce-f2bc-4e42-a3e8-9ca4e950bfc1,109.0,Remedy,PLAY,NaN,1.630264e+12,0.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:10:18.606,0,8
14,Q5D01u10sfOs4gOLpgdsbmlrk8q2,6-b0a49c9f-b5ba-4cc1-8cdf-71a5227eabb0,109.0,Remedy,PAUSE,NaN,1.630264e+12,120679.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:12:19.534,0,8
26,Q5D01u10sfOs4gOLpgdsbmlrk8q2,7-770f6cce-572f-4ebb-9d23-d2ff9ddcec66,109.0,Remedy,NaN,PAUSE,1.630264e+12,120789.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:12:19.646,0,8
27,Q5D01u10sfOs4gOLpgdsbmlrk8q2,8-640e7027-312c-422b-814d-042ab84cafe2,440.0,Hey! Baby,NaN,PLAY,1.630270e+12,120967.0,"'60s Gold, Disc 1",Bruce Channel,143935.0,Hey! Baby,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 20:39:52.560,0,8
15,Q5D01u10sfOs4gOLpgdsbmlrk8q2,9-6802a95c-c2d7-45bb-a3d4-f632fd7132ce,440.0,Hey! Baby,PLAY,NaN,1.630270e+12,0.0,"'60s Gold, Disc 1",Bruce Channel,143935.0,Hey! Baby,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 20:39:52.648,0,8


In [14]:
for idx in range(1, len(user_behavior_data)):
  user = user_behavior_data.loc[idx, "user_id"]
  song = user_behavior_data.loc[idx, "song_id"]
  event_seek_position= user_behavior_data.loc[idx, "event_seek_position_in_milliseconds"]
  event_type = user_behavior_data.loc[idx, "player_event_type"]
  trigger_time = user_behavior_data.loc[idx, "event_current_time_in_milliseconds"]
  listening_time = 0
  if user != user_behavior_data.loc[idx-1, "user_id"]:
    continue
  elif song != user_behavior_data.loc[idx-1, "song_id"]:
    continue
  else:
    if event_type != "PLAY" and user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY": #this is covering same song pause-play
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)
    if (event_type == "PAUSE" or event_type == "NEXT" or event_type == "PREV") and (user_behavior_data.loc[idx-1, "player_event_type"] == "SEEK" or user_behavior_data.loc[idx-1, "player_event_type"] == "PLAY"):#when idx-1 is play and idx is either play or next or prev
        listening_time = trigger_time - user_behavior_data.loc[idx-1, "event_current_time_in_milliseconds"]
        user_behavior_data.loc[idx, "listening_time"] = listening_time/(1000*60)

C:\Users\chowd\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\chowd\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [15]:
user_behavior_data

,user_id,record_id,song_id,song_name,player_event_type,ui_event_type,event_current_time_in_milliseconds,event_seek_position_in_milliseconds,song_album_name,song_artist_name,song_duration,song_name,audio_volume_data,event_current_time_ms_to_date,min_volume,curent_volume,listening_time
16,Q5D01u10sfOs4gOLpgdsbmlrk8q2,0-a335579d-23a8-43fc-aa59-2fef8c89feb3,NaN,NaN,NaN,VOLUME_DOWN,1.630264e+12,0.0,NaN,NaN,NaN,NaN,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:06:12.167,0,8,NaN
17,Q5D01u10sfOs4gOLpgdsbmlrk8q2,1-da4b4f93-1619-4acb-b528-639c68df1d42,142.0,When You Wish Upon A Star,NaN,PLAY,1.630264e+12,0.0,Dream With Me,Jackie Evancho,221440.0,When You Wish Upon A Star,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:06:36.321,0,8,NaN
5,Q5D01u10sfOs4gOLpgdsbmlrk8q2,2-575fd343-a771-448e-9fcd-ad5c686ef5e6,142.0,When You Wish Upon A Star,PLAY,NaN,1.630264e+12,0.0,Dream With Me,Jackie Evancho,221440.0,When You Wish Upon A Star,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:06:36.486,0,8,NaN
12,Q5D01u10sfOs4gOLpgdsbmlrk8q2,3-f29bdaba-924f-44fb-ae29-478c4c072135,143.0,Nella Fantasia,PLAY,NaN,1.630264e+12,0.0,Dream With Me,Jackie Evancho,258534.0,Nella Fantasia,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:10:18.159,0,8,NaN
25,Q5D01u10sfOs4gOLpgdsbmlrk8q2,4-12388d9e-dd3b-4b86-964b-3258f79a754f,109.0,Remedy,NaN,PLAY,1.630264e+12,433.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:10:18.472,0,8,NaN
13,Q5D01u10sfOs4gOLpgdsbmlrk8q2,5-c193f0ce-f2bc-4e42-a3e8-9ca4e950bfc1,109.0,Remedy,PLAY,NaN,1.630264e+12,0.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:10:18.606,0,8,NaN
14,Q5D01u10sfOs4gOLpgdsbmlrk8q2,6-b0a49c9f-b5ba-4cc1-8cdf-71a5227eabb0,109.0,Remedy,PAUSE,NaN,1.630264e+12,120679.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:12:19.534,0,8,2.015467
26,Q5D01u10sfOs4gOLpgdsbmlrk8q2,7-770f6cce-572f-4ebb-9d23-d2ff9ddcec66,109.0,Remedy,NaN,PAUSE,1.630264e+12,120789.0,25,Adele,245473.0,Remedy,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 19:12:19.646,0,8,NaN
27,Q5D01u10sfOs4gOLpgdsbmlrk8q2,8-640e7027-312c-422b-814d-042ab84cafe2,440.0,Hey! Baby,NaN,PLAY,1.630270e+12,120967.0,"'60s Gold, Disc 1",Bruce Channel,143935.0,Hey! Baby,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 20:39:52.560,0,8,NaN
15,Q5D01u10sfOs4gOLpgdsbmlrk8q2,9-6802a95c-c2d7-45bb-a3d4-f632fd7132ce,440.0,Hey! Baby,PLAY,NaN,1.630270e+12,0.0,"'60s Gold, Disc 1",Bruce Channel,143935.0,Hey! Baby,"VolumeData [currentVolumeLevel=8, volumeMax=15...",2021-08-29 20:39:52.648,0,8,NaN


In [16]:
path = r'A:\MUSER\data\cleaned_data\mergedFiles\dummy\analysis_Code'

user_behavior_data.to_csv(os.path.join(path,'analysis_3.csv'), index=False, encoding='utf-8-sig')